# This demo shows how to do a general search for similar images

This demo is setup using python3

Follow up more information on product webpage: https://help.aliyun.com/document_detail/113827.html

## flow:
1. imagesearch_product to get product category (my_image_search with instanceName_Product)
2. imagesearch_general to upload images with detected category
3. imagesearch_general to find similar images (my_image_search with instanceName_General)

In [ ]:
#setup envionrmnet for python depedency if necessary
! pip3 install aliyun-python-sdk-imagesearch #--upgrade 
! pip3 install pillow #--upgrade 

# Get access credential from environemnt setting

In [ ]:
import os
accessKeyId = os.environ.get('accessKeyId', 'not_set')
accessKeySecret = os.environ.get('accessKeySecret', 'not_set')
instanceName_Product = os.environ.get('instanceName_Product', 'not_set')
instanceName_General = os.environ.get('instanceName_General', 'not_set')

print(instanceName_Product, instanceName_General)

In [ ]:
# setup image search connection
product = "ImageSearch"
region = 'cn-shanghai' # , ap-southeast-1(sg)

domain = 'imagesearch.cn-shanghai.aliyuncs.com' # 
endpoint = 'imagesearch.cn-shanghai.aliyuncs.com' # 

from aliyunsdkcore.profile import region_provider
from aliyunsdkcore.client import AcsClient
region_provider.add_endpoint(product, region, domain)
client  = AcsClient(accessKeyId, accessKeySecret, region)
requestClient = client

# Load supporting function

In [ ]:
%load_ext autoreload
%autoreload 2
from myutil import *

# Demo for single image upload and detect object category

In [ ]:
image_name = '1.jpg' 
image_folder = './image' 
image_path = os.path.abspath(image_folder+'/'+image_name)

print("image path:", image_name, image_path)
img1 = my_image_preview(image_path, '0,0,0,0', 'unknown category')
img1

In [ ]:
# test to upload a single image
image_name = '1.jpg'
image_path = os.path.abspath(image_folder+'/'+image_name)
result = my_image_search_for_category_detection_display(requestClient, instanceName_Product, image_path)
print(image_path)

# Upload all images into imagesearch_General with detected category

In [ ]:
image_folder = './image' 
images = list_images(image_folder)
#pprint(images)

for image_path in images:
    image_name = images[image_path]
    #image_path = './image/1.jpg'
    
    r = my_image_search_for_category_detection(requestClient, instanceName_Product, image_path)
    cate_id = r['cate_id']
    cate_desc = r['cate_desc']
    obj_region = r['obj_region']
    r = my_image_upload_for_similarity_search(requestClient, endpoint, instanceName_General, image_name, image_name, image_path, cate_id, cate_desc, obj_region)
    
    time.sleep(1)

# Search similar images

In [ ]:
#result = my_image_search_for_similarity(requestClient, instanceName_General, './image/1.jpg')


In [ ]:
image_folder = './image' 
images = list_images(image_folder)
#pprint(images)

for image_path in images:
    image_name = images[image_path]
    r = my_image_search_for_similarity(requestClient, instanceName_General, image_path)   
    time.sleep(1)

# Delete all images being uploaded

In [ ]:
image_folder = './image' 
images = list_images(image_folder)
#pprint(images)

for image_path in images:
    image_name = images[image_path]
    request = DeleteImageRequest.DeleteImageRequest()
    request.set_endpoint(endpoint)
    request.set_InstanceName(instanceName_Product)
    request.set_ProductId(image_name)
    response = client.do_action_with_exception(request)
    print(image_name)
    print(json.loads(response))
    time.sleep(1)